In [4]:
import pandas as pd
import hmac
import hashlib
import requests
from io import StringIO

In [5]:
checksums = {'f8cc462996f8c328ec4fad35a842b8e07efa2497b263410ccff7e29429efa586', 'cc72438b990d7dffbfab383c38486696302c8257882524a43bd5845e70cb2447'}

In [6]:
url = 'https://www.ncei.noaa.gov/access/monitoring/climate-at-a-glance/statewide/time-series/13/pcp/1/9/1895-2024.csv?base_prd=true&begbaseyear=1901&endbaseyear=2000'
response = requests.get(url)
checksum = hashlib.sha256(response.text.encode('utf-8')).hexdigest()

if checksum in checksums:
    print('Checksum verification passed.')
    df_ncei = pd.read_csv(StringIO(response.text), skiprows=4)
    print(df_ncei.head())
else:
    print('Checum verification failed.')

Checksum verification passed.
     Date  Value  Anomaly
0  189509   3.12    -0.55
1  189609   3.97     0.30
2  189709   1.97    -1.70
3  189809   2.62    -1.05
4  189909   0.96    -2.71


In [7]:
endpoint = 'https://data.iowa.gov/resource/tw78-ziwj.csv?$limit=583174'
response = requests.get(endpoint)
checksum = hashlib.sha256(response.text.encode('utf-8')).hexdigest()

if checksum in checksums:
    print('Checksum verification passed.')
    df_accidents = pd.read_csv(StringIO(response.text))
    print(df_accidents.head())
else:
    print('Checksum verification failed.')

Checksum verification passed.


/var/folders/j7/fswh0x992tzfxgs898w7vv9w0000gn/T/ipykernel_4473/898798598.py:7: DtypeWarning: Columns (17,23,35) have mixed types. Specify dtype option on import or set low_memory=False.
  df_accidents = pd.read_csv(StringIO(response.text))


   casenumber   lecasenum            date_of_crash month_crash    day_crash  \
0  2014781102   14-000003  2014-01-01T00:00:00.000  01-January  4-Wednesday   
1  2014777414  2014000131  2014-01-01T00:00:00.000  01-January  4-Wednesday   
2  2014778079  2014000054  2014-01-01T00:00:00.000  01-January  4-Wednesday   
3  2014777620   14-000033  2014-01-01T00:00:00.000  01-January  4-Wednesday   
4  2014778188  K14-000050  2014-01-01T00:00:00.000  01-January  4-Wednesday   

  timestr        hour            dot_district        city_name    county_name  \
0   00:42  0000 Hours    District 1 (Central)             AMES          STORY   
1   14:00  1400 Hours  District 4 (Southwest)  WEST DES MOINES         DALLAS   
2   09:38  0900 Hours  District 4 (Southwest)              NaN  POTTAWATTAMIE   
3   19:03  1900 Hours  District 5 (Southeast)              NaN          HENRY   
4   17:11  1700 Hours  District 5 (Southeast)           KEOKUK            LEE   

   ... injuries majinjury mininjury po

In [8]:
## cleaning df_accidents
date_time = pd.to_datetime(df_accidents['date_of_crash'])
df_accidents['year'] = date_time.dt.year
df_accidents = df_accidents[['year', 'enviro_conditions', 'light_conditions', 'surface_conditions', 'weather_conditions',
                             'crash_severity', 'fatalities', 'injuries', 'majinjury', 'mininjury', 'possinjury', 'unkinjury', 'propdmg']]

## cleaning df_ncei
df_ncei['year'] = df_ncei['Date'].apply(lambda x: int(str(x)[:4]))
df_ncei.rename(columns={'Value':'precipitation'}, inplace=True)
df_ncei = df_ncei[['year', 'precipitation']]